## To run the pipeline:
##### in general: test_mode is a boolean argument, that speeds up all processes by using pre-existing data if found.
##### for a real analysis or to overwrite pre-existing files, you should set test_mode to False

In [ ]:
from core.meta import meta_interface

##### overwrite: bool whether to overwrite the meta.yaml at the given project_name or if existing appending the project_name by _01

In [ ]:
d = meta_interface(
    project_config_filepath="test_data/project_config.yaml",
    project_name="test_data",
    overwrite=True,
)

In [ ]:
d.add_recording_config("test_data/Server_structure/Calibrations/220922/recording_config_220922.yaml")

In [ ]:
d.initialize_meta_config()

Found 1 recordings at recording day 220922!


#### creates undistorted images per cam and reads metadata
##### test_mode = True doesnt save the undistorted images

In [ ]:
d.create_recordings(test_mode=True)

AttributeError: 'Reader' object has no attribute 'count_frames'

#### runs synchronisation and DLC analysis
##### test_mode=True doesn't overwrite already existing synchronized/analysed files
##### synchronize_only=True doesn't DLC analyse files
##### verbose=True prints the duration of one recording analysis

In [ ]:
d.synchronize_recordings(verbose=True, test_mode=True)

#### creates undistorted images per cam and reads metadata
##### test_mode = True doesnt save the undistorted images

In [ ]:
d.create_calibrations(
    ground_truth_config_filepath="test_data/ground_truth_config_only_corners.yaml", test_mode=True
)

#### runs synchronisation
##### test_mode=True doesn't overwrite already existing synchronized videos

In [ ]:
d.synchronize_calibrations(test_mode=True)

In [ ]:
d.exclude_markers(all_markers_to_exclude_config_path = "test_data/markers_to_exclude_config.yaml", verbose=False)

#### Anipose calibration
##### calibrate_optimal = True uses calibration_validation for repeated calibrations
##### verbose > 1 shows aniposelib output. verbose > 0 shows Calibration_validation output. verbose = 0 doesnt show any output

In [ ]:
d.calibrate(verbose=2, calibrate_optimal=True, test_mode=True)

In [ ]:
d.triangulate_recordings(test_mode=True)

In [ ]:
d.normalize_recordings(normalization_config_path="test_data/normalization_config.yaml", test_mode = True)

In [ ]:
d.objects["triangulation_recordings_objects"][
    "206_F2-63_220922_OTE"
].create_triangulated_video("test_video", "test_data/video_plotting_config_OTE.yaml")

#### Add metadata to database

In [ ]:
# create empty database
import pandas as pd

df = pd.DataFrame(
    {},
    columns=[
        "recording",
        "date",
        "session_id",
        "paradigm",
        "subject_id",
        "group_id",
        "batch",
        "trial_id",
    ],
)
df.to_csv("test.csv", index=False)

In [ ]:
# add recording, date, paradigm, subject id
d.add_triangulated_csv_to_database("test.csv")

In [ ]:
# add group id, session id, batch, trial id
def add_database_metadata(recording, overwrite=False) -> None:
    old_df = pd.read_csv(database, dtype="str")
    if (
        old_df.loc[
            old_df["recording"] == recording,
            ["session_id", "group_id", "batch", "trial_id"],
        ]
        .isna()
        .any(axis=1)
        .values[0]
        or overwrite
    ):
        subject_id = old_df.loc[old_df["recording"] == recording, "subject_id"].values[
            0
        ]
        recording_date = old_df.loc[old_df["recording"] == recording, "date"].values[0]

        try:
            batch = groups.loc[groups["subject_id"] == subject_id, "batch"].values[0]
        except IndexError:
            raise ValueError(f"Found no group identity for {subject_id}!")
        group = groups.loc[groups["subject_id"] == subject_id, "group_id"].values[0]
        try:
            session = (
                time.loc[time["batch"] == str(batch),]
                .loc[time["date"] == recording_date, "session_id"]
                .values[0]
            )
        except IndexError:
            raise ValueError(f"Found no session identity for {subject_id}!")
        trial_id = (
            time.loc[time["batch"] == batch,]
            .loc[time["date"] == recording_date, "trial_id"]
            .values[0]
        )

        old_df.loc[
            old_df["recording"] == recording,
            ["group_id", "batch", "session_id", "trial_id"],
        ] = (group, batch, session, trial_id)
        old_df.to_csv(database, index=False)
    else:
        print("metadata was already in csv!")


time = pd.read_csv("time.csv", dtype="str")
groups = pd.read_csv("groups.csv", dtype="str")
database = "test.csv"

for recording in pd.read_csv(database, dtype="str")["recording"].values:
    add_database_metadata(recording, overwrite=False)

In [ ]:
pd.read_csv("test.csv")

### The meta .yaml file contains all information stored in d.meta

In [ ]:
d.export_meta_to_yaml(filepath="test_data/test_data.yaml")

In [ ]:
# not fully supported at the moment
d.load_meta_from_yaml(filepath="test_data/test_data.yaml")

#### To inspect the files, we recommend https://www.treedoc.org/, which creates a tree-like structured view for .yaml files